In [0]:
use catalog identifier(:catalog);
use schema identifier(:schema);

create table if not exists identifier(:table) (
  id bigint generated always as identity,
  job_name string not null comment 'Name of the job used to run the task.',
  task_collection string not null comment 'Used to group tasks into collections.',
  src_type string comment 'Source database type.',
  src_catalog string not null comment 'Name of the source catalog',
  src_schema string not null comment 'Name of the source schema.',
  src_table string not null comment 'Name of the source table.',
  sink_catalog string not null comment 'Name of the sink catalog',
  sink_schema string not null comment 'Name of the sink schema.',
  sink_table string not null comment 'Name of the sink table.',
  primary_key array<string> not null comment 'Array of columns used as the primary key.',
  sink_cluster_cols array<string> comment 'Array of columns used for Liquid Clustering',
  load_type string not null comment 'Specified is load is full or incremental.',
  load_partitioned boolean comment 'Specifies if load is partitioned or not.',
  select_list string not null comment 'List of columns or expressions used to select data from the source.',
  watermark_col_name string comment 'Name of the column used to track incremental loads.',
  watermark_col_type string comment 'Data type of the column used to track incremental loads.',
  watermark_col_start_value string comment 'Watermark starting value for incremental loads.',
  partition_col string comment 'Name of column used for performing partitioned loads.',
  partition_size_mb int comment 'Individual partition size in MB for partitioned loads (E.g., 256, 512).',
  task_enabled boolean not null comment 'Flag used to enable or disable the task.'
);

alter table identifier(:table) drop constraint if exists valid_load_type;
alter table identifier(:table) add constraint valid_load_type check (load_type in ('full', 'incremental'));

alter table identifier(:table) drop constraint if exists valid_src_type;
alter table identifier(:table) add constraint valid_src_type check (src_type in ('sqlserver', 'oracle', 'postgresql', 'redshift', 'synapse', 'delta'));

-- Ensure incremental tasks have required metadata
alter table identifier(:table) drop constraint if exists valid_incremental_metadata;
alter table identifier(:table) add constraint valid_incremental_metadata check (
  (load_type = 'incremental' and array_size(primary_key) > 0 and watermark_col_name is not null and watermark_col_type is not null and watermark_col_start_value is not null) or (load_type = 'full' and array_size(primary_key) = 0 and watermark_col_name is null and watermark_col_type is null and watermark_col_start_value is null)
);